# read README file first

In [11]:
import pandas as pd
import tensorflow as tf
import numpy as np
import pandas as pd
import numpy as np
import cv2
import os
import re

In [45]:
path_train_csv="/home/chirag/Downloads/Kaggle_project/train.csv"
img_dir_all_data="/home/chirag/Downloads/Kaggle_project/My_project/train"
train_df=pd.read_csv(path_train_csv)


def split_Bbox(value):
    reg = np.array(re.findall("([0-9]+[.]?[0-9]*)", value))
    if len(reg) == 0:
        reg = [-1, -1, -1, -1]
        print(reg)
    return reg

def csv_manipulation_and_filtering(df):
    df['xmin'] = 0
    df['ymin'] = 0
    df['xmax'] = 0
    df['ymax'] = 0
    df["width"]=600
    df["height"]=600
    
    df=df.rename(columns={"image_id": "filename"})
    df=df.rename(columns={"source": "class"})
    df[['xmin', 'ymin', 'xmax', 'ymax']] = np.stack(df['bbox'].apply(lambda value: split_Bbox(value)))
    df.drop(columns=['bbox'], inplace=True)
        
    df['class']= "wheat_head_detected"
    df["filename"]=df['filename'].astype(str)+'.jpg'
    
    df['xmin'] = df['xmin'].astype(np.float)
    df['ymin'] = df['ymin'].astype(np.float)
    df['xmax'] = df['xmax'].astype(np.float)
    df['ymax'] = df['ymax'].astype(np.float)
    df['xmax']=df['xmin']+df['xmax']
    df['ymax']=df['ymin']+df['ymax']
  
        
    return df


df=csv_manipulation_and_filtering(train_df)
df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,b6ab77fd7.jpg,600,600,wheat_head_detected,834.0,222.0,890.0,258.0
1,b6ab77fd7.jpg,600,600,wheat_head_detected,226.0,548.0,356.0,606.0
2,b6ab77fd7.jpg,600,600,wheat_head_detected,377.0,504.0,451.0,664.0
3,b6ab77fd7.jpg,600,600,wheat_head_detected,834.0,95.0,943.0,202.0
4,b6ab77fd7.jpg,600,600,wheat_head_detected,26.0,144.0,150.0,261.0


In [35]:
## code for finding images which do not have BBOXs ## 

uniq=df["filename"].unique()
from PIL import Image
import glob
fileList = []
count=0
for root, dirs, files in os.walk(img_dir_all_data, topdown=False):
    for name in files:
        if name.endswith('.jpg'):
            name_of_img=os.path.splitext(name)[0]
            count+=1
            if name_of_img not in uniq:
                fileList.append(name_of_img) 
                
                
len(fileList)   

# REMOVE THESE IMAGES FROM DATASET !!

3421

In [36]:
import sys
sys.path.append("/home/chirag/Downloads/models-master/research/slim/")
from object_detection.legacy import train

In [37]:
df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,b6ab77fd7.jpg,600,600,wheat_head_detected,834.0,222.0,56.0,36.0
1,b6ab77fd7.jpg,600,600,wheat_head_detected,226.0,548.0,130.0,58.0
2,b6ab77fd7.jpg,600,600,wheat_head_detected,377.0,504.0,74.0,160.0
3,b6ab77fd7.jpg,600,600,wheat_head_detected,834.0,95.0,109.0,107.0
4,b6ab77fd7.jpg,600,600,wheat_head_detected,26.0,144.0,124.0,117.0


In [38]:
test_csv=df[0:47]
train_csv=df[47:]

In [39]:
test_csv.tail()

,filename,width,height,class,xmin,ymin,xmax,ymax
42,b6ab77fd7.jpg,600,600,wheat_head_detected,959.0,869.0,23.0,31.0
43,b6ab77fd7.jpg,600,600,wheat_head_detected,600.0,723.0,221.0,81.0
44,b6ab77fd7.jpg,600,600,wheat_head_detected,575.0,146.0,72.0,121.0
45,b6ab77fd7.jpg,600,600,wheat_head_detected,945.0,193.0,57.0,45.0
46,b6ab77fd7.jpg,600,600,wheat_head_detected,216.0,440.0,46.0,20.0


In [40]:
train_csv.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
47,b53afdf5c.jpg,600,600,wheat_head_detected,988.0,781.0,36.0,96.0
48,b53afdf5c.jpg,600,600,wheat_head_detected,331.0,863.0,70.0,95.0
49,b53afdf5c.jpg,600,600,wheat_head_detected,47.0,226.0,65.0,46.0
50,b53afdf5c.jpg,600,600,wheat_head_detected,682.0,974.0,108.0,50.0
51,b53afdf5c.jpg,600,600,wheat_head_detected,63.0,988.0,156.0,36.0


In [41]:
train_output_path="/home/chirag/Downloads/Kaggle_project/My_project/train.record"
test_output_path="/home/chirag/Downloads/Kaggle_project/My_project/test.record"

test_img_dir="/home/chirag/Downloads/Kaggle_project/images/test/"

filtered_img_path="/home/chirag/Downloads/Kaggle_project/images/train/"

 # Run this script 2 times for creating "train.record" and "test.record" 

In [46]:
#chnage your path , pass train and test dataframe

In [43]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict


def class_text_to_int(row_label):
    if row_label == 'wheat_head_detected':
        return 1
    else:
        None


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main(_):
    writer = tf.python_io.TFRecordWriter(train_output_path)
    path = os.path.join(filtered_img_path)
    
    examples=train_csv
    
    grouped = split(examples, 'filename')
    
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path = os.path.join(train_output_path)
    print('Successfully created the TFRecords: {}'.format(output_path))


if __name__ == '__main__':
    tf.app.run()


Successfully created the TFRecords: /home/chirag/Downloads/Kaggle_project/My_project/train.record


SystemExit: 

/home/chirag/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [48]:

import functools
import json
import os
import tensorflow as tf
from tensorflow.contrib import framework as contrib_framework

from object_detection.builders import dataset_builder
from object_detection.builders import graph_rewriter_builder
from object_detection.builders import model_builder
from object_detection.legacy import trainer
from object_detection.utils import config_util

tf.logging.set_verbosity(tf.logging.INFO)
flags = tf.app.flags


FLAGS = flags.FLAGS

# Create TRAINING folder in dir
# copy this (faster_rcnn_resnet50_pets.config) and paste in this folder
# do changes accordingly 
# create  lable.pbtxt file in this folder and put path in "faster_rcnn_resnet50_pets"
# run this Cell and it will create many models in folders, it takes alot of time

traning_dir="/home/Downloads/models-master/research/object_detection/tranining"
Pipeline_config_path ="/home/Downloads/models-master/research/object_detection/tranining/faster_rcnn_resnet50_pets.config"

@contrib_framework.deprecated(None, 'Use object_detection/model_main.py.')

def main(_):
  if FLAGS.task == 0: tf.gfile.MakeDirs(traning_dir)
  if Pipeline_config_path:
    configs = config_util.get_configs_from_pipeline_file(
        Pipeline_config_path)
    if FLAGS.task == 0:
      tf.gfile.Copy(Pipeline_config_path,
                    os.path.join(traning_dir, 'pipeline.config'),
                    overwrite=True)
  else:
    configs = config_util.get_configs_from_multiple_files(
        model_config_path=FLAGS.model_config_path,
        train_config_path=FLAGS.train_config_path,
        train_input_config_path=FLAGS.input_config_path)
    if FLAGS.task == 0:
      for name, config in [('model.config', FLAGS.model_config_path),
                           ('train.config', FLAGS.train_config_path),
                           ('input.config', FLAGS.input_config_path)]:
        tf.gfile.Copy(config, os.path.join(FLAGS.train_dir, name),
                      overwrite=True)

  model_config = configs['model']
  train_config = configs['train_config']
  input_config = configs['train_input_config']

  model_fn = functools.partial(
      model_builder.build,
      model_config=model_config,
      is_training=True)

  def get_next(config):
    return dataset_builder.make_initializable_iterator(
        dataset_builder.build(config)).get_next()

  create_input_dict_fn = functools.partial(get_next, input_config)

  env = json.loads(os.environ.get('TF_CONFIG', '{}'))
  cluster_data = env.get('cluster', None)
  cluster = tf.train.ClusterSpec(cluster_data) if cluster_data else None
  task_data = env.get('task', None) or {'type': 'master', 'index': 0}
  task_info = type('TaskSpec', (object,), task_data)

  # Parameters for a single worker.
  ps_tasks = 0
  worker_replicas = 1
  worker_job_name = 'lonely_worker'
  task = 0
  is_chief = True
  master = ''

  if cluster_data and 'worker' in cluster_data:
    # Number of total worker replicas include "worker"s and the "master".
    worker_replicas = len(cluster_data['worker']) + 1
  if cluster_data and 'ps' in cluster_data:
    ps_tasks = len(cluster_data['ps'])

  if worker_replicas > 1 and ps_tasks < 1:
    raise ValueError('At least 1 ps task is needed for distributed training.')

  if worker_replicas >= 1 and ps_tasks > 0:
    # Set up distributed training.
    server = tf.train.Server(tf.train.ClusterSpec(cluster), protocol='grpc',
                             job_name=task_info.type,
                             task_index=task_info.index)
    if task_info.type == 'ps':
      server.join()
      return

    worker_job_name = '%s/task:%d' % (task_info.type, task_info.index)
    task = task_info.index
    is_chief = (task_info.type == 'master')
    master = server.target

  graph_rewriter_fn = None
  if 'graph_rewriter_config' in configs:
    graph_rewriter_fn = graph_rewriter_builder.build(
        configs['graph_rewriter_config'], is_training=True)

  trainer.train(
      create_input_dict_fn,
      model_fn,
      train_config,
      master,
      task,
      FLAGS.num_clones,
      worker_replicas,
      FLAGS.clone_on_cpu,
      ps_tasks,
      worker_job_name,
      is_chief,
      traning_dir,
      graph_hook_fn=graph_rewriter_fn)


if __name__ == '__main__':
  tf.app.run()


Instructions for updating:
Use object_detection/model_main.py.


W0526 01:10:12.764706 140498178537280 deprecation.py:323] From /home/chirag/anaconda3/lib/python3.7/site-packages/absl/app.py:250: main (from __main__) is deprecated and will be removed in a future version.
Instructions for updating:
Use object_detection/model_main.py.


W0526 01:10:12.766551 140498178537280 deprecation_wrapper.py:119] From /home/chirag/anaconda3/lib/python3.7/site-packages/object_detection/utils/config_util.py:94: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



Instructions for updating:
Use `tf.cast` instead.


W0526 01:10:12.788052 140498178537280 deprecation.py:323] From /home/chirag/anaconda3/lib/python3.7/site-packages/object_detection/anchor_generators/grid_anchor_generator.py:59: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


...

I0526 01:10:56.503250 140498178537280 learning.py:785] Finished training! Saving model to disk.


Instructions for updating:
Use standard file APIs to delete files with this prefix.


W0526 01:10:57.077833 140498178537280 deprecation.py:323] From /home/chirag/anaconda3/lib/python3.7/site-packages/tensorflow/python/training/saver.py:960: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.
/home/chirag/anaconda3/lib/python3.7/site-packages/tensorflow/python/summary/writer/writer.py:386: UserWarning: Attempting to use a closed FileWriter. The operation will be a noop unless the FileWriter is explicitly reopened.
  warnings.warn("Attempting to use a closed FileWriter. "


SystemExit: 

/home/chirag/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# run this command in terminal

In [ ]:
# MAKE SURE, YOU ARE IN "/home/models-master/research/object_detection"
# in export_inference_graph.py, don't forget to put 
#                                "sys.path.append("/home/chirag/Downloads/models-master/research/slim/")"  

In [ ]:
# "export_inference_graph.py " download this from my repo

In [ ]:
#  python export_inference_graph.py 
#     --input_type image_tensor 
#     --pipeline_config_path tranining/faster_rcnn_resnet50_pets.config  
#     --trained_checkpoint_prefix tranining/model.ckpt-4310 
#     --output_directory inference_graph


In [ ]:
## NOW YOU WILL HAVE .PB file in "inference_graph" folder